# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from nithiya_api_keys import g_key

In [3]:
# Configure gmaps
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [4]:
# # Create aiport dataframe
# airport_df = pd.read_csv('../Resources/Airport_Output.csv')
# airport_df.dropna(inplace = True) 
# airport_df.head()


vacation_df = pd.read_csv('nithiya_output_data/cities.csv')

vacation_df

,city,lat,long,temp,temp_max,humidity,cloudiness,wind speed
0,Longyearbyen,78.2186,15.6401,40.84,40.84,65,75,13.80
1,Manta,-0.9500,-80.7333,73.18,73.18,88,90,6.91
2,Fairbanks,64.8378,-147.7164,54.34,58.82,88,20,1.01
3,Hong Kong,22.2855,114.1577,87.71,89.33,76,23,1.01
4,Tura,25.5198,90.2201,75.83,75.83,95,82,8.21
...,...,...,...,...,...,...,...,...
583,Egvekinot,66.3167,-179.1667,34.16,34.16,92,97,3.60
584,Tuktoyaktuk,69.4541,-133.0374,35.28,35.28,89,100,5.97
585,Dzilam González,21.2833,-88.9333,74.01,74.01,97,100,7.90
586,Santa Cruz del Sur,20.7147,-77.9956,78.57,78.57,85,64,7.52


In [5]:
vacation_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 588 entries, 0 to 587
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   city        588 non-null    object 
 1   lat         588 non-null    float64
 2   long        588 non-null    float64
 3   temp        588 non-null    float64
 4   temp_max    588 non-null    float64
 5   humidity    588 non-null    int64  
 6   cloudiness  588 non-null    int64  
 7   wind speed  588 non-null    float64
dtypes: float64(5), int64(2), object(1)
memory usage: 36.9+ KB


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [6]:
# # Store latitude and longitude in locations
# locations = airport_df[["Lat", "Lng"]]

# # Fill NaN values and convert to float
# rating = airport_df["Airport Rating"].astype(float)

locations = vacation_df[["lat", "long"]]
humidity = vacation_df["humidity"]

In [ ]:
# pip install ipywidgets

In [7]:
!jupyter nbextension enable --py widgetsnbextension --sys-prefix

Config option `kernel_spec_manager_class` not recognized by `EnableNBExtensionApp`.
Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK


In [8]:
# Plot Heatmap
%matplotlib inline
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [9]:
ideal_df = vacation_df[(vacation_df['temp'] >= 70) & (vacation_df['temp'] <= 80)]
ideal_df = ideal_df[ideal_df['wind speed'] <= 10]
ideal_df = ideal_df[ideal_df['cloudiness'] <= 5]
# ideal_df = ideal_df[ideal_df['humidity'] <= 70]
hotel_df = ideal_df
hotel_df

,city,lat,long,temp,temp_max,humidity,cloudiness,wind speed
52,Havelock,34.8791,-76.9013,71.60,75.94,90,5,5.79
77,Kletnya,53.3897,33.2171,77.16,77.16,41,0,7.78
96,Galveston,29.3669,-94.9669,77.83,80.96,90,3,9.64
103,Capim Grosso,-11.3811,-40.0128,74.10,74.10,61,4,7.02
131,Homestead,25.4687,-80.4776,75.58,79.02,93,1,0.00
297,Vangaindrano,-23.3500,47.6000,73.47,73.47,66,2,4.72
322,Vila Velha,-20.3297,-40.2925,71.85,75.96,77,0,1.99
326,San Patricio,28.0170,-97.5169,76.69,81.84,89,3,6.60
327,Pandamatenga,-18.5278,25.6270,78.53,78.53,20,2,4.47
360,Tateyama,34.9833,139.8667,72.27,72.99,80,5,8.97


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [19]:
hotel_df['Hotel Name'] = ""
hotel_df['rating'] = ""
hotel_df

,city,lat,long,temp,temp_max,humidity,cloudiness,wind speed,Hotel Name,opening_hours,rating
52,Havelock,34.8791,-76.9013,71.60,75.94,90,5,5.79,,open_now,
77,Kletnya,53.3897,33.2171,77.16,77.16,41,0,7.78,,open_now,
96,Galveston,29.3669,-94.9669,77.83,80.96,90,3,9.64,,open_now,
103,Capim Grosso,-11.3811,-40.0128,74.10,74.10,61,4,7.02,,open_now,
131,Homestead,25.4687,-80.4776,75.58,79.02,93,1,0.00,,open_now,
297,Vangaindrano,-23.3500,47.6000,73.47,73.47,66,2,4.72,,,
322,Vila Velha,-20.3297,-40.2925,71.85,75.96,77,0,1.99,,,
326,San Patricio,28.0170,-97.5169,76.69,81.84,89,3,6.60,,,
327,Pandamatenga,-18.5278,25.6270,78.53,78.53,20,2,4.47,,,
360,Tateyama,34.9833,139.8667,72.27,72.99,80,5,8.97,,,


In [24]:
for index, row in hotel_df.iterrows():
    try:
        
        base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

        params = {
        "keyword": "hotel",
        "radius": 5000,
        "key": g_key,
        }


        lat = row['lat']
        lng = row['long']

        params['location'] = f"{lat}, {lng}"

        hotel_data = requests.get(base_url, params=params).json()
#         print(hotel_data)
        
        hotel_df.loc[index, "Hotel Name"] = hotel_data["results"][0]["name"]
        hotel_df.loc[index, "rating"] = hotel_data["results"][0]["rating"]
    except IndexError:
        
        hotel_df.loc[index, "Hotel Name"] = "NaN"
        hotel_df.loc[index, "rating"] = "NaN"
        
hotel_df

,city,lat,long,temp,temp_max,humidity,cloudiness,wind speed,Hotel Name,opening_hours,rating
52,Havelock,34.8791,-76.9013,71.60,75.94,90,5,5.79,Holiday Inn Express & Suites Havelock NW-New B...,open_now,4.2
77,Kletnya,53.3897,33.2171,77.16,77.16,41,0,7.78,Gostinitsa-Otel' Kletnya,open_now,4.1
96,Galveston,29.3669,-94.9669,77.83,80.96,90,3,9.64,"Candlewood Suites Texas City, an IHG Hotel",open_now,4.1
103,Capim Grosso,-11.3811,-40.0128,74.10,74.10,61,4,7.02,Hotel Veloso,open_now,3.8
131,Homestead,25.4687,-80.4776,75.58,79.02,93,1,0.00,Holiday Inn Express & Suites Florida City-Gate...,open_now,3.7
297,Vangaindrano,-23.3500,47.6000,73.47,73.47,66,2,4.72,Hotel Fanilo,,2
322,Vila Velha,-20.3297,-40.2925,71.85,75.96,77,0,1.99,Hotel Senac Ilha do Boi,,4.7
326,San Patricio,28.0170,-97.5169,76.69,81.84,89,3,6.60,"Motel 6 Sinton, TX",,3.4
327,Pandamatenga,-18.5278,25.6270,78.53,78.53,20,2,4.47,Touch of Africa Safari Lodge,,4.5
360,Tateyama,34.9833,139.8667,72.27,72.99,80,5,8.97,Flower Splash (Hospitality In Full Bloom),,4.2


In [25]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{city}</dd>
<dt>rating</dt><dd>{rating}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["lat", "long"]]

In [23]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)
fig

# Display figure


Figure(layout=FigureLayout(height='420px'))